# Running in parallel

While these tutorials are typically meant to be visualized in serial, `viskex` also runs in parallel. To exemplify this, we will show a parallel case using `ipyparallel`.

First of all, we start a `ipyparallel` MPI cluster with 2 processes.

In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines="MPI", profile="mpi", n=2)
cluster.start_and_connect_sync()

The jupyter magic `%%px` will run the following cells on the `ipyparallel` cluster.

In [ ]:
%%px
import firedrake  # noqa: E402
import mpi4py.MPI  # noqa: E402

In [ ]:
%%px
import viskex  # noqa: E402

To confirm that we are running in parallel, we can print the rank of the current process and the total number of processes.

In [ ]:
%%px
comm_world = mpi4py.MPI.COMM_WORLD
print(f"rank = {comm_world.rank}, size = {comm_world.size}")

Generate a mesh on `MPI_COMM_WORLD` of the unit square by dividing each edge of the square in 6 segments.

In [ ]:
%%px
square_world = firedrake.UnitSquareMesh(
    6, 6, comm=comm_world, distribution_parameters={"partitioner_type": "simple"})

Plot the mesh defined on `MPI_COMM_WORLD`. Each rank will plot its local cells, and their neighbors (called halos in `firedrake`).

In [ ]:
%%px
viskex.firedrake.plot_mesh(square_world)

For comparison, we can generate a similar mesh on `MPI_COMM_SELF`. Each rank will have its own copy of the whole mesh.

In [ ]:
%%px
comm_self = mpi4py.MPI.COMM_SELF
square_self = firedrake.UnitSquareMesh(
    6, 6, comm=comm_self, distribution_parameters={"partitioner_type": "simple"})

Plot the mesh defined on `MPI_COMM_SELF`. Each rank will produce a plot which is visually the same, since each rank has its own copy of the same mesh.

In [ ]:
%%px
viskex.firedrake.plot_mesh(square_self)

We finally stop the `ipyparallel` cluster. Note that when running with `Run -> Run all cells` all interactive plots will be closed when the kernel executes this cell.

In [ ]:
cluster.stop_cluster_sync()